In [ ]:
%%capture

# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(1400, 900))
# display.start()


# # 의사 디스플레이 설정
# import os
# if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
#     !bash ../xvfb start
#     %env DISPLAY=:1
    
import gym
from gym.wrappers import Monitor
import matplotlib.pyplot as plt

In [1]:
import xgboost as xgb
import cv2

In [12]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
    """
    Taken from https://github.com/eleurent/highway-env
  
    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                    </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

def wrap_env(env, video_path):
    env = Monitor(env, video_path, force=True)
    return env

In [2]:
from gym import envs
for idx, env_name in enumerate(envs.registry.all()):
    print(idx, env_name)

0 EnvSpec(entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=27000, order_enforce=True, kwargs={'game': 'tetris', 'obs_type': 'rgb', 'repeat_action_probability': 0.25, 'full_action_space': False, 'frameskip': 4}, namespace='ALE', name='Tetris', version=5)
1 EnvSpec(entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=27000, order_enforce=True, kwargs={'game': 'tetris', 'obs_type': 'ram', 'repeat_action_probability': 0.25, 'full_action_space': False, 'frameskip': 4}, namespace='ALE', name='Tetris-ram', version=5)
2 EnvSpec(entry_point='gym.envs.atari:AtariEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=27000, order_enforce=True, kwargs={'game': 'adventure', 'obs_type': 'rgb', 'repeat_action_probability': 0.25, 'full_action_space': False, 'frameskip': 4}, namespace='ALE', name='Adventure', version=5)
3 EnvSpec(entry_point='gym.envs.atari:AtariEnv', reward_thresho

In [5]:
%%capture
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!pip install tensorflow==1.15.2

In [3]:
%%capture
# !pip install stable-baselines[mpi]==2.10.0 box2d-py box2d-kengz

import logging
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [8]:
import gym
import numpy as np

# from stable_baselines import DQN
from stable_baselines.common import atari_wrappers
from stable_baselines.common.cmd_util import make_atari_env
from stable_baselines.common.vec_env import VecFrameStack

- Pong 환경 생성

In [8]:
!wget http://www.atarimania.com/roms/Roms.rar
!unrar x /content/Roms.rar /content
!python -m atari_py.import_roms .

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Extracting  /content/ROMS/Bank Heist (Hack) (32 in 1) (Bit Corporation) (R320).bin       3%  OK 
Extracting  /content/ROMS/Bank Heist (Unknown) (PAL) [a].bin               3%  OK 
Extracting  /content/ROMS/Bank Heist (Unknown) (PAL).bin                   3%  OK 
Extracting  /content/ROMS/Barnstorming (1982) (Activision, Steve Cartwright) (AX-013) ~.bin       3%  OK 
Extracting  /content/ROMS/Barnstorming (CCE).bin                           3%  OK 
Extracting  /content/ROMS/Barnstorming (Unknown) (PAL) [a].bin             3%  OK 
Extracting  /content/ROMS/Barnstorming (Unknown) (PAL).bin                 3%  OK 
Extracting  /content/ROMS/Barnstorming - Die tollkeuhnen Flieger (1982) (Activision, Steve Cartwright - Ariola) (EAX-013, PAX-013 - 711 013-720) (PAL).bin       3%  OK 
Extracting  /content/ROMS/Base Attack (1983) (Home Vision - Gem International Corporation - VDI) (VCS8311

In [9]:
env = atari_wrappers.make_atari('PongNoFrameskip-v4')
env = atari_wrappers.wrap_deepmind(env)

- Pong 환경에서 유효한 행동 리스트 출력

In [10]:
print(env.action_space.n)
print(env.unwrapped.get_action_meanings())

6
['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


In [13]:
import random

env = wrap_env(env, video_path="./env_video_sample_1")
state = env.reset()
done = False
steps = 0
max_steps = 500
while not done and steps < max_steps:
    steps += 1
    action = random.choice(range(env.action_space.n))
    if steps % 20 != 0:
        print(action, end=", ")
    else:
        print(action, end="\n")
    next_state, reward, done, _ = env.step(action)

# Close the video recorder
env.close()

NameError: name 'Monitor' is not defined

In [ ]:
show_videos(video_path="./env_video_sample_1")

### 모델학습

In [19]:
def imageresize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    return np.array(img)

def state_to_xgbinput(img, pre_model):
    resized_img = imageresize(img)
    input = pre_model(resized_img.reshape([-1,224,224,3])).numpy()
    return input

In [30]:
def image_to_input(img):
    s2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    s3 = cv2.resize(s2, dsize=(80, 80), interpolation=cv2.INTER_AREA)
    s4 = s3.reshape(1,-1)
    return s4

def state_to_xgbinput(img, pre_model):
    resized_img = imageresize(img)
    input = pre_model(resized_img.reshape([-1,224,224,3])).numpy()
    return input

In [31]:
def make_model(states, actions, model_name):
    # x = np.concatenate(states, axis = 0).reshape([-1,4])
    x = np.array(states).reshape([-1,6400])
    y = np.array(actions).reshape([-1,1])
    dtrain = xgb.DMatrix(x, y)
    if model_name == None:
        model = xgb.train(params, dtrain)
    else:
        model = xgb.train(params, dtrain, xgb_model=model_name)
    return model

def get_action(xgbmodel, input_state,):
    input = image_to_input(input_state)
    input = xgb.DMatrix(input)
    out = xgbmodel.predict(input)
    a = out.argmax()
    # if out > 0.5:
    #     a = 1
    # else:
    #     a = 0
    return a

def get_action_egreedy(xgbmodel, input_state,eps):
    if np.random.choice([True, False],p = [eps,1-eps]):
        a = np.random.choice([0, 1,2,3,4,5])
    else:
        a = get_action(xgbmodel,input_state)
    return a

In [ ]:
params = {
    "tree_method": "hist", 
    # "objective": 'binary:logistic',
    # "objective": 'binary:logistic',
    "objective": 'multi:softprob',
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "single_precision_histogram" : True,
    "num_class": 6,
          }

# env = atari_wrappers.make_atari('PongNoFrameskip-v4')
# env = atari_wrappers.wrap_deepmind(env)
# env = wrap_env(env, video_path="./env_video_sample_2")
env = gym.make('PongNoFrameskip-v4')

s = env.reset()
input = image_to_input(s)
xgb_clf = make_model(input,[[5]], model_name=None)
xgb_clf.save_model("model_pong2.json")

epsilon = 1
eps_min = 0.1


total_r_list = []
# states_list = []
# actions_list = []


avg = -21
pass_f = 3


for _ in range(1000):
    act_pass_frame = pass_f  
    train_x = []
    train_y = []
    states = []
    actions = []
    total_r = 0
    s = env.reset()
    a = get_action_egreedy(xgb_clf, s,epsilon)

    states.append(image_to_input(s))
    actions.append(a)
    while True:
        ns, r, done, _ = env.step(a)
        # total_r += r
        total_r += r

        if r == -1:
          states = []
          actions = []

        elif r == 1:
          epsilon = epsilon*0.99
          train_x.extend(states)
          train_y.extend(actions)
        #   xgb_clf = make_model(train_x,train_y,"model_pong2.json")
        #   xgb_clf = make_model(train_x,train_y, None)
          states = []
          actions = []



        

        if done:
            print(total_r)
            print(epsilon)
            total_r_list.append(total_r)
            # states_list.append(states)
            # actions_list.append(actions)
            break
        else:
            s = ns
            if act_pass_frame == 0 :
                a = get_action_egreedy(xgb_clf, s,epsilon)
                # print(a)
                act_pass_frame = pass_f
            act_pass_frame -= 1
            states.append(image_to_input(s))
            actions.append(a)
    
    if len(train_y) != 0:
        xgb_clf = make_model(train_x,train_y,"model_pong2.json")
        xgb_clf.save_model("model_pong2.json")
         





    # train_x = []
    # train_y = []

    # for i in range(len(total_r_list)):
    #     if total_r_list[i] > avg:
    #         train_x.extend(states_list[i])
    #         train_y.extend(actions_list[i])
    # if len(train_y) != 0:
    #     xgb_clf = make_model(train_x,train_y,"model_pong2.json")

env.close()